In [1]:
import scipy.io as sio
import numpy as np
mt = sio.loadmat('/home/maria/Documents/EnsemblePursuit/BehaviorData/spont_M161025_MP030_20161120.mat')
spks = mt['Fsp']                   # neurons by timepoints
med = mt['med']                 # cell centers (X Y Z)

### behavioral measures
runSpeed = mt['beh']['runSpeed']    # running speed
# get motion SVDs (time x components)
motionSVD=np.array(mt['beh'][0]['face'][0]['motionSVD'][0][0])

In [2]:
print(spks.shape)

(10164, 15761)


In [3]:
import sys
sys.path.append("..")
from EnsemblePursuitModule.EnsemblePursuitNumpy import EnsemblePursuitNumpy
import numpy as np
import time
options_dict={'seed_neuron_av_nr':100,'min_assembly_size':8}
#ep_pt=EnsemblePursuitNumpy(n_ensembles=300,lambd=0.005,options_dict=options_dict)
ep_np=EnsemblePursuitNumpy(n_ensembles=150,lambd=0.01,options_dict=options_dict)
U,V=ep_np.fit_transform(spks)

KeyboardInterrupt: 

In [ ]:
bundle={'U':U,'V':V}
#np.save('beh_ep.npy',bundle)


In [ ]:
U=np.load('beh_ep.npy',allow_pickle=True)[()]['U']
print(U)
V=np.load('beh_ep.npy',allow_pickle=True)[()]['V']

In [ ]:
from Pipelines.SingleMousePipeline import ModelPipelineSingleMouse
pipeline=ModelPipelineSingleMouse('/home/maria/Documents/EnsemblePursuit/BehaviorData/','spont_M161025_MP030_20161120.mat','EnsemblePursuit_numpy',150)
pipeline.variance_explained_across_neurons(U,V)

In [ ]:
from utils import zscore
import matplotlib.pyplot as plt
def variance_explained_across_neurons(spks,U,V):
    '''
    The coefficient R^2 is defined as (1 - u/v), where u is the residual sum of squares
    ((y_true - y_pred) ** 2).sum() and v is the total sum of squares
    ((y_true - y_true.mean()) ** 2).sum().
    '''
    #Fetch the original data and convert it into the same form as what goes into the
    #matrix factorization model
    X=zscore(spks.T).T
    u=[]
    v=[]
    approx=U@V.T
    for j in range(X.shape[0]):
        u_j=((X[j,:]-approx[j,:])**2).sum()
        v_j=((X[j,:]-np.mean(X[j,:]))**2).sum()
        u.append(u_j)
        v.append(v_j)
    u=np.array(u)
    v=np.array(v)
    plt.plot(-np.divide(u,v)+1)
    plt.title('Variance explained across neurons')
    plt.show()
    print('Total variance explained, averaged over neurons is:',(1-np.mean(u)/np.mean(v)))

variance_explained_across_neurons(spks,U,V)


In [ ]:
beh=motionSVD
motSVD=motionSVD.T
print(motSVD.shape)
print(V.shape)
parea =np.array(mt['beh'][0]['pupil'][0]['area'][0][0])
S=spks
import matplotlib.pyplot as plt
tbin = 1.0
NT = motSVD.shape[1]
nt = int(np.floor(NT/tbin))
motSVD -= motSVD.mean(axis=1)[:,np.newaxis]
parea  -= np.nanmean(parea)
parea[np.isnan(parea)] = 0
tbin = int(tbin)

beh = np.reshape(motSVD[:,:nt*tbin], (motSVD.shape[0], nt, tbin)).mean(axis=-1)
pup = np.reshape(parea[:nt*tbin], (nt, tbin)).mean(axis=-1)

# using this format bin S
Sbin = np.reshape(S[:,:nt*tbin], (S.shape[0], nt, tbin)).mean(axis=-1)
# split into train-test
# * use interleaved segments *
nsegs = 20
nlen  = nt/nsegs
ninds = np.linspace(0,nt-nlen,nsegs).astype(int)
itest = (ninds[:,np.newaxis] + np.arange(0,nlen*0.25,1,int)).flatten()
itrain = np.ones(nt, np.bool)
itrain[itest] = 0

plt.plot(itrain)
plt.show()

#### PREDICT USING PUPIL WITH LINEAR REGRESSION
V=V.T
#pup =np.array(mt['beh'][0]['pupil'][0]['area'][0][0]) 
A = np.matmul(pup[itrain], V[:,itrain].T)/(pup**2).sum()

Vpredp = np.matmul(A[:,np.newaxis], pup[itest][np.newaxis,:])

varexp_pupil = 1 - ((Vpredp - V[:,itest])**2).sum(axis=1)/(V[:,itest]**2).sum(axis=1)

#### PREDICT USING BEHAVIOR PC'S 
## regularized linear regression from behavior to neural PCs
covM = np.matmul(beh[:,itrain], beh[:,itrain].T)
lam = 1e5 # regularizer
covM += lam*np.eye(beh.shape[0])
A = np.linalg.solve(covM, np.matmul(beh[:,itrain], V[:,itrain].T))

Vpred = np.matmul(A.T, beh[:,itest])

varexp = 1 - ((Vpred - V[:,itest])**2).sum(axis=1)/(V[:,itest]**2).sum(axis=1)

In [ ]:
fig=plt.figure(figsize=(12,3))

ipc = 1 ### which PC to plot

ax = fig.add_axes([0.05,.05,.75,.95])
ax.plot(V[ipc,itest], label='neural PC')
ax.plot(Vpred[ipc], color='k', label='face PC prediction')
ax.set_title('PC %d'%ipc)
ax.set_xlabel('time')
ax.set_ylabel('activity')
ax.legend()

ax = fig.add_axes([0.9,.05, .2, .8])
ax.semilogx(np.arange(1,varexp.size+1), varexp, color='k')
ax.scatter(ipc+1, varexp[ipc],marker='x',color='r',s=200, lw=4, zorder=10)
ax.semilogx(np.arange(1,varexp.size+1), varexp_pupil, color=[0.,.5,0])
ax.text(1,0,'pupil',color=[0,.5,0])
ax.text(10,0.2,'motion SVD')
ax.set_xlabel('PC')
ax.set_ylabel('fraction variance explained')
ax.set_title('PC %d, varexp=%0.2f'%(ipc,varexp[ipc]))
plt.show()